In [2]:
# Import library yang diperlukan
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
from fastai.vision.all import *


ModuleNotFoundError: No module named 'fastai'

In [ ]:
# Set folder dataset dan hasil
dataset_path = "asl"
output_folder = "hasil_mediapine"
os.makedirs(output_folder, exist_ok=True)



In [ ]:
# Inisialisasi MediaPipe untuk deteksi tangan
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5)

In [ ]:
# Fungsi untuk memproses gambar dengan MediaPipe
def process_image(image_path, output_folder):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Menggambar landmark tangan di gambar
            mp.solutions.drawing_utils.draw_landmarks(
                image_rgb, hand_landmarks, mp_hands.HAND_CONNECTIONS
            )

        # Simpan gambar hasil
        output_path = os.path.join(output_folder, os.path.basename(image_path))
        cv2.imwrite(output_path, cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR))
        return True
    return False

In [ ]:
# Proses semua gambar dalam folder dataset
for letter in os.listdir(dataset_path):
    letter_folder = os.path.join(dataset_path, letter)
    if os.path.isdir(letter_folder):
        for img_name in os.listdir(letter_folder):
            img_path = os.path.join(letter_folder, img_name)
            process_image(img_path, output_folder)

In [ ]:
# Load dataset untuk FastAI
path = Path(output_folder)
files = get_image_files(path)

In [ ]:
# Siapkan label berdasarkan folder
labels = [f.name for f in files]
y = [f.parent.name for f in files]

In [ ]:
# Membagi dataset menjadi train dan valid
splits = RandomSplitter(valid_pct=0.2)(files)
dls = ImageDataLoaders.from_name_re(path, labels, valid_pct=0.2, seed=42)



In [ ]:
# Membuat dan melatih model
learn = cnn_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(10)


In [ ]:
# Simpan model
learn.export("asl_model.pkl")




In [ ]:
# Fungsi untuk prediksi huruf isyarat
def predict_sign(image_path):
    img = PILImage.create(image_path)
    pred, pred_idx, probs = learn.predict(img)
    return pred, probs[pred_idx].item()




In [ ]:
# Uji model dengan gambar baru
test_image_path = "path/to/test_image.jpg"  # Ganti dengan path gambar yang ingin diuji
predicted_label, probability = predict_sign(test_image_path)
print(f"Predicted: {predicted_label}, Probability: {probability:.2f}")

In [ ]:
# streamlit_app.py

import streamlit as st
from fastai.vision.all import *
from PIL import Image

# Load model
learn = load_learner('asl_model.pkl')

# Judul aplikasi
st.title('Pendeteksi Bahasa Isyarat Amerika')

# Unggah gambar
uploaded_file = st.file_uploader("Pilih gambar tangan...", type=['jpg', 'png', 'jpeg'])
if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Gambar yang diunggah', use_column_width=True)

    # Prediksi
    pred, pred_idx, probs = learn.predict(image)
    st.write(f'Prediksi: {pred}, Probabilitas: {probs[pred_idx]:.2f}')
